CSE 404 Project

### Project Overview

### Data Cleaning and Preprocessing

##### Exploratory Data Analysis (EDA)
- Check the structure of the dataset:
  - Verify column names and data types
  - Ensure consistency in stock symbols
- Identify missing values:
  - If missing values exist in **Open/Close**: Fill with the average of previous and next day's values. 
  - If missing values exist in **Volume**: Fill with the median for a stable estimare or zero if it represents a non-trading day.
- Identify duplicate rows:
  - If duplicate trading days exist for the same stock, drop them.

##### Split dataset into training, test, and validation set
- Use **70/15/15 split** for training, testing, and validation


##### Feature Scaling 
- Scale features using **StandardScaler**

In [7]:
import numpy as np
import pandas as pd

In [8]:
stocks= pd.read_csv("stocks.csv")
stocks.head()

,Close_AAPL,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,Close_GOOG,High_GOOG,Low_GOOG,Open_GOOG,Volume_GOOG,...,Close_^IXIC,High_^IXIC,Low_^IXIC,Open_^IXIC,Volume_^IXIC,Close_^VIX,High_^VIX,Low_^VIX,Open_^VIX,Volume_^VIX
0,0.842151,0.846382,0.765036,0.789017,535796800.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.771149,0.832276,0.761274,0.814408,512377600.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.782433,0.831806,0.774910,0.780553,778321600.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.714723,0.805004,0.714723,0.798421,767972800.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.748578,0.759864,0.718485,0.726008,460734400.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Logistic Regression

In [ ]:
# Load the CSV file
file_path = "stocks.csv"
df = pd.read_csv(file_path)

# Display basic info and first few rows
df.info(), df.head()

### Neural Network Model:

### Evaluation